In [1]:
import pandas as pd

In [2]:
import os
from pathlib import Path

import pandas as pd

from asme.data.datamodule.converters import CsvConverter
from asme.data.datamodule.util import read_csv


In [27]:
input_dir = Path("/Users/lisa/recommender/datasets/ml-1m-raw")
output_file = Path("/Users/lisa/recommender/datasets/ml-1m-extended/ml-1m-extendend.csv")

In [28]:
base_config= "/Users/lisa/recommender/src/plp_paper/local-ml-1m.yaml"
expanded_config= "/Users/lisa/recommender/src/plp_paper/local-ml-1m-plp.yaml"

## Download and split Dataset with Default Values

In [4]:
from asme.core.utils.run_utils import load_config, create_container
import typer
import torch

torch.multiprocessing.set_start_method('spawn', force=True)

def create_original_data(config_file: Path = typer.Argument(..., help='the path to the config file', exists=True)) -> None:
    config_file_path = Path(config_file)
    config = load_config(config_file_path)
    #Creates the dataset:
    container = create_container(config)

create_original_data(Path(base_config))

Preprocessing dataset:
Unpacking dataset.
Applying preprocessing step 'Converting to CSV' (1/2)
Skipping this step since dry run is available.
Applying preprocessing step 'Generating ratio split. (0.8/0.1/0.1).' (2/2)
Property 'features.item.tokenizer.vocabulary.file' was not found in the configuration. Inferring it to '/Users/lisa/recommender/datasets/ml-1m/ml-1m/ratio_split-0.8_0.1_0.1/ml-1m.vocabulary.title.txt'.
Your module contains a submodule named <embedding> that does not report its hyperparameters, consider adding @save_hyperparameters annotation.


File /Users/lisa/recommender/datasets/ml-1m/ml-1m/ml-1m.zip already downloaded


In [122]:
input_dir = Path("/Users/lisa/recommender/datasets/ml-1m/ml-1m/ratio_split-0.8_0.1_0.1")
output_dir = Path("/Users/lisa/recommender/datasets/ml-1m/ml-1m-overview/ratio_split-0.8_0.1_0.1")
name = "ml-1m"

import os
def create_extended_movielens_data(input_dir, output_dir, name, stage):
    file_type = ".csv"
    encoding = "latin-1"
    delimiter = "\t"
    item_df = read_csv(input_dir, f'{name}.{stage}', file_type, "\t", header=0, encoding=encoding)
    item_df["cat_title"] = item_df["title"]
    item_df["old_title"] = item_df["title"]
    item_df["item_id_type"] = 1
    page_df_mod = item_df.copy()
    page_df_mod["old_title"] = page_df_mod["title"]
    page_df_mod["title"] = "OVERVIEW-PAGE"
    page_df_mod["cat_title"] = page_df_mod["genres"]
    page_df_mod["item_id_type"] = 0
    page_df_mod["rating"] = -1
    page_df_mod["year"] = 0
    item_df['original_order'] = item_df.groupby(['userId', 'timestamp']).cumcount() + 1
    page_df_mod['original_order'] = page_df_mod.groupby(['userId', 'timestamp']).cumcount() + 1
    item_df = pd.concat([item_df,page_df_mod], ignore_index=True)
    item_df = item_df.sort_values(["userId","timestamp","original_order","item_id_type"])
    item_df = item_df[['userId', 'rating', 'timestamp', 'gender', 'age',
                       'occupation', 'title', 'genres', 'year', 'user_all', 'cat_title', 'item_id_type']]
    os.makedirs(output_dir, exist_ok=True)
    item_df.to_csv(f'{output_dir}/{name}.{stage}{file_type}', sep=delimiter, index=False)

for stage in ["test","train","validation"]:
    create_extended_movielens_data(input_dir,output_dir,name,stage)


In [29]:
vocabulary_file_plain ="/Users/lisa/recommender/datasets/ml-1m/ml-1m/ratio_split-0.8_0.1_0.1/ml-1m.vocabulary.title.txt"
vocabulary_file_target ="/Users/lisa/recommender/datasets/ml-1m/ml-1m-overview/ratio_split-0.8_0.1_0.1/ml-1m-extended.vocabulary.cat_title.txt"

In [45]:
reduced_vocabulary = pd.read_csv(vocabulary_file_plain, header= None, sep="\t")
full_vocabulary = pd.read_csv(vocabulary_file_target, header= None, sep="\t")

In [54]:
vocab_join = full_vocabulary.merge(reduced_vocabulary, on= 0, how='inner')["1_x"].astype(int).tolist()

In [55]:
vocab_join

[0,
 1,
 2,
 4,
 6,
 8,
 10,
 12,
 14,
 16,
 18,
 20,
 22,
 24,
 26,
 28,
 30,
 32,
 34,
 36,
 40,
 42,
 44,
 46,
 48,
 50,
 52,
 54,
 56,
 58,
 60,
 62,
 64,
 66,
 68,
 70,
 72,
 74,
 76,
 78,
 80,
 82,
 84,
 86,
 88,
 90,
 92,
 94,
 96,
 97,
 99,
 100,
 101,
 103,
 105,
 107,
 109,
 111,
 113,
 115,
 116,
 117,
 118,
 119,
 121,
 123,
 125,
 127,
 129,
 131,
 132,
 133,
 134,
 136,
 138,
 140,
 142,
 144,
 146,
 147,
 149,
 151,
 153,
 155,
 157,
 159,
 161,
 163,
 165,
 167,
 169,
 171,
 173,
 175,
 177,
 179,
 181,
 183,
 185,
 187,
 189,
 191,
 192,
 194,
 195,
 196,
 198,
 200,
 202,
 204,
 206,
 207,
 208,
 210,
 212,
 214,
 216,
 218,
 220,
 222,
 224,
 226,
 228,
 230,
 232,
 234,
 236,
 238,
 240,
 242,
 244,
 246,
 248,
 250,
 252,
 254,
 256,
 258,
 260,
 262,
 264,
 266,
 268,
 270,
 272,
 274,
 276,
 278,
 280,
 282,
 284,
 286,
 288,
 290,
 292,
 294,
 296,
 298,
 300,
 302,
 304,
 306,
 308,
 310,
 312,
 314,
 315,
 316,
 317,
 319,
 321,
 323,
 325,
 327,
 329,
 331,
 

In [7]:
from typing import Callable, Any, List


def load_file_with_item_ids(path: Path) -> List[int]:
    """
    loads a file containing item ids into a list
    :param path: the path of the file
    :return:
    """
    items = _load_file_line_by_line(path, int)
    sorted(items)
    return items


def _load_file_line_by_line(path: Path, line_converter: Callable[[str], Any]) -> List[Any]:
    with open(path) as item_file:
        return [line_converter(line) for line in item_file.readlines()]